In [ ]:
import numpy as np

In [ ]:
np.load('')

In [ ]:
id_data = pyblp.build_id_data(T=10, J=20, F=10)

In [ ]:
integration = pyblp.Integration('product', 9)

In [ ]:
simulation = pyblp.Simulation(
   product_formulations=(
       pyblp.Formulation('1 + prices + x'),
       pyblp.Formulation('0 + x'),
       pyblp.Formulation('0 + x + z')
   ),
   beta=[1, -2, 2],
   sigma=1,
   gamma=[1, 4],
   product_data=id_data,
   integration=integration,
   seed=0
)

In [ ]:
simulation_results = simulation.replace_endogenous()

In [ ]:
problem = simulation_results.to_problem()

In [ ]:
results = problem.solve(
    sigma=0.5 * simulation.sigma,
    pi=0.5 * simulation.pi,
    beta=[None, 0.5 * simulation.beta[1], None],
    method='1s',
    optimization=pyblp.Optimization('return')
)

In [ ]:
pyblp_passthrough = results.compute_passthrough()
pyblp_passthrough

In [ ]:
marginal_costs = pd.Series(np.squeeze(simulation_results.costs))
marginal_costs_tilde = marginal_costs.copy()
marginal_costs_tilde[0] = marginal_costs[0] + .001
simulation_results_1 = simulation.replace_endogenous(costs=marginal_costs_tilde)

In [ ]:
prices_high = simulation_results_1.product_data['prices']
prices_high

In [ ]:
marginal_costs = pd.Series(np.squeeze(simulation_results.costs))
marginal_costs_tilde = marginal_costs.copy()
marginal_costs_tilde[0] = marginal_costs[0] - .001
simulation_results_1 = simulation.replace_endogenous(costs=marginal_costs_tilde)

In [ ]:
prices_low = simulation_results_1.product_data['prices']
prices_low

In [ ]:
numerical_passthrough = (prices_high - prices_low) / .002
numerical_passthrough

In [ ]:
results.compute_passthrough()

First, get all the common elements for market t:

In [ ]:
from pyblp.utilities.basics import extract_matrix, get_indices
import scipy.linalg

# pull out all relevant information for market t
t = 0
index_t = np.where(results.problem.products['market_ids'] == t)[0]
shares = extract_matrix(results.problem.products, "shares").astype(float)
shares_t = shares[index_t]
print(len(shares_t))
# get jacobian and hessian matrices from pyblp results
ds_dp = results.compute_demand_jacobians()
d2s_dp2_t = results.compute_demand_hessians(market_id=t)

# get retailer response and ownership matrices
retailer_response_matrix = ds_dp[index_t]
retailer_response_matrix = retailer_response_matrix[:, ~np.isnan(retailer_response_matrix).all(axis=0)]
retailer_ownership_matrix = pyblp.build_ownership(product_data=simulation_results.product_data)
retailer_ownership_matrix_t = retailer_ownership_matrix[index_t]

Try to construct passthrough using our code (Vilas Boas formula)

In [ ]:
J = len(shares_t)
g = np.zeros((J, J))
markups_t = -scipy.linalg.inv(retailer_ownership_matrix_t * retailer_response_matrix) @ shares_t
for j in range(J):
    g[j] = np.transpose(markups_t) @ (retailer_ownership_matrix_t * d2s_dp2_t[:, :, j])
print(g[0][0])

# solve for derivatives of all prices with respect to the wholesale prices
H = np.transpose(retailer_ownership_matrix_t * retailer_response_matrix)
G = retailer_response_matrix + H + g
passthrough = scipy.linalg.inv(G) @ H
passthrough

Now, using the procedure from pyblp. Their ownership_matrix is our retailer_ownership_matrix_t, their ds_dp is retailer_response_matrix

In [ ]:
# compute passthrough matrix using pyblp notation
capital_delta = -retailer_ownership_matrix_t * retailer_response_matrix
capital_delta_derivatives = -retailer_ownership_matrix_t[..., None] * d2s_dp2_t  # the None adds extra axis to multiply with hessian
capital_delta_inverse = scipy.linalg.inv(capital_delta)

# compute the passthrough inverse
passthrough_inverse = np.zeros((J, J), np.float64)
for j in range(J):
    passthrough_inverse[:, [j]] = (
        capital_delta_inverse @ capital_delta_derivatives[..., j] @ capital_delta_inverse @ shares_t
    )
passthrough_inverse += np.eye(J) - capital_delta_inverse @ retailer_response_matrix

# invert to get passthrough matrix
passthrough = scipy.linalg.inv(passthrough_inverse)
passthrough

Our (retailer_ownership_matrix_t * d2s_dp2_t[:, :, j]) is equal to their capital_delta_derivatives[..., j] but times -1.

In [ ]:
print(capital_delta_derivatives[..., j][0])
print((retailer_ownership_matrix_t * d2s_dp2_t[:, :, j])[0])

Trying to manipulate our code to get the result from pyblp passthrough

In [ ]:
J = len(shares_t)
g = np.zeros((J, J), np.float64)
markups_t = results.compute_markups(market_id=t)
for j in range(J):
    g[:, [j]] = (retailer_ownership_matrix_t * d2s_dp2_t[j, :, :]) @ capital_delta_inverse @ shares_t

# solve for derivatives of all prices with respect to the wholesale prices
H = np.transpose(retailer_ownership_matrix_t * retailer_response_matrix)
G = retailer_response_matrix + H + g
passthrough_matrix = scipy.linalg.inv(G) @ H
print(passthrough_matrix)

# compute passthrough matrix using pyblp notation
capital_delta = -retailer_ownership_matrix_t * retailer_response_matrix
capital_delta_derivatives = -retailer_ownership_matrix_t[..., None] * d2s_dp2_t  # the None adds extra axis to multiply with hessian
capital_delta_inverse = scipy.linalg.inv(capital_delta)

# compute the passthrough inverse
passthrough_inverse = np.zeros((J, J), np.float64)
for j in range(J):
    passthrough_inverse[:, [j]] = (
        capital_delta_inverse @ capital_delta_derivatives[..., j] @ capital_delta_inverse @ shares_t
    )
# print(passthrough_inverse[0])
passthrough_inverse += np.eye(J) - capital_delta_inverse @ retailer_response_matrix
# print(passthrough_inverse[0])

# invert to get passthrough matrix
passthrough = scipy.linalg.inv(passthrough_inverse)

print(passthrough)

In [ ]:
print((capital_delta_inverse @ shares_t)[0])
prices = extract_matrix(results.problem.products, "prices").astype(float)
prices_t = prices[index_t]
print(markups_t[0] * prices_t[0])
print((-scipy.linalg.inv(retailer_ownership_matrix_t * retailer_response_matrix) @ shares_t)[0])

In [ ]:
# for j in range(J):
#     outer_sum = 0
#     for k in range(J):
#         inner_sum = 0
#         for i in range(J):
#             inner_sum += retailer_ownership_matrix_t[i, j] * d2s_dp2_t[i, j, k] * markups_t[i]
#         outer_sum += ds_dp[j, k] + inner_sum + retailer_ownership_matrix_t[k, j] * ds_dp[k, j]
#     g[j, k] = outer_sum
# g

J = len(shares_t)
inner_sum = 0
for i in range(J):
    inner_sum += retailer_ownership_matrix_t[i, 0] * d2s_dp2_t[0, 0, i] * markups_t[i]
print(inner_sum[0])
outer_sum = ds_dp[0, 0] + inner_sum + retailer_ownership_matrix_t[0, 0] * np.transpose(ds_dp)[0, 0]
print(outer_sum[0])

In [ ]:
delta_p_inv = 2 + (1 / retailer_response_matrix[0, 0]) * (d2s_dp2_t[0, 0, 0]) * (markups_t[0])
delta_p_inv

In [ ]:
2 + scipy.linalg.inv(retailer_response_matrix) @ (d2s_dp2_t[:, :, 0]) * (markups_t[0])

In [ ]:
delta_p_inv = (1 / retailer_response_matrix[0, 0]) * (d2s_dp2_t[0, 0, 0]) * (markups_t[0])
delta_p_inv

In [ ]:
s1 = shares_t[0]
s2 = shares_t[1]

outer = ((1 - s1)**2 * (1 - s2)**2) / (1 - s1 - s2)
first = 1 / (1 - s2)
second = (s1 * s2) / (1 - s1)**2
third = (s1 * s2) / (1 - s2)**2
fourth = 1 / (1 - s1)
print(outer * first, outer * second, outer * third, outer * fourth)

In [ ]:
2 - (1 / retailer_response_matrix[0, 0])**2 * (d2s_dp2_t[0, 0, 0]) * s1

In [ ]:
2 * (retailer_response_matrix[0, 0]) - (1 / (retailer_response_matrix[0, 0])) * (d2s_dp2_t[0, 0, 0]) * s1

In [ ]:
%timeit
import numpy as np

J = 20
myarray = np.ones((J, J), dtype=object)
myvector = np.array((1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 16, 17, 18, 19, 20))
def function1():
    mylist = [None] * J
    for j in range(J):
        mylist[j] = myvector @ myarray

def function2():
    mybigarray = np.zeros((J, J))
    for j in range(J):
        mybigarray[j, :] = myvector @ myarray


In [ ]:
import numpy as np
import pandas as pd
import pyblp
import pyRVtest

pyblp.options.digits = 2
pyblp.options.verbose = False
pyRVtest.options.digits = 2
pyRVtest.__version__

# load data
product_data = pd.read_csv(pyblp.data.NEVO_PRODUCTS_LOCATION)
agent_data = pd.read_csv(pyblp.data.NEVO_AGENTS_LOCATION)

# estimate demand
pyblp_problem = pyblp.Problem(
    product_formulations=(
        pyblp.Formulation('0 + prices ', absorb='C(product_ids)'),
        pyblp.Formulation('1 + prices + sugar + mushy'),
    ),
    agent_formulation=pyblp.Formulation('0 + income + income_squared + age + child'),
    product_data=product_data,
    agent_data=agent_data
)
pyblp_results = pyblp_problem.solve(
  sigma=np.diag([0.3302, 2.4526, 0.0163, 0.2441]),
  pi=[
      [5.4819,   0.0000,  0.2037, 0.0000],
      [15.8935, -1.2000,  0.0000, 2.6342],
      [-0.2506,  0.0000,  0.0511, 0.0000],
      [1.2650,   0.0000, -0.8091, 0.0000]
  ],
  method='1s',
  optimization=pyblp.Optimization('bfgs', {'gtol': 1e-5})
)

In [ ]:
testing_problem = pyRVtest.Problem(
    cost_formulation = (
        pyRVtest.Formulation('0 + sugar', absorb = 'C(firm_ids)' )
    ),
    instrument_formulation = (
        pyRVtest.Formulation('0 + demand_instruments0 + demand_instruments1')
    ),
    model_formulations = (
        pyRVtest.ModelFormulation(model_downstream='bertrand', ownership_downstream='firm_ids'),
        pyRVtest.ModelFormulation(model_downstream='monopoly', ownership_downstream='firm_ids')
    ),
    product_data = product_data,
    demand_results = pyblp_results
)

In [ ]:
def kappa_specification(f, g):
    kappa = np.array([
        [1, .2, .3, 0, 0, 0, 0],
        [0, 1, .3, 0, 0, 0, 0],
        [0, .2, 1, 0, 0, 0, 0],
        [0, .2, .3, 1, 0, 0, 0],
        [0, .2, .3, 0, 1, 0, 0],
        [0, .2, .3, 0, 0, 1, 0],
        [0, .2, .3, 0, 0, 0, 1]
    ])
    if f == g:
       return 1
    return kappa[f, g]

# build_ownership(product_data, kappa_specification=kappa_specification)
testing_problem = pyRVtest.Problem(
    cost_formulation = (
        pyRVtest.Formulation('0 + sugar', absorb = 'C(firm_ids)' )
    ),
    instrument_formulation = (
        pyRVtest.Formulation('0 + demand_instruments0 + demand_instruments1')
    ),
    model_formulations = (
        pyRVtest.ModelFormulation(
            model_downstream='bertrand',
            ownership_downstream='firm_ids',
            kappa_specification_downstream=kappa_specification
        ),
        pyRVtest.ModelFormulation(model_downstream='monopoly', ownership_downstream='firm_ids')
    ),
    product_data = product_data,
    demand_results = pyblp_results
)

In [ ]:
testing_results = testing_problem.solve(
    demand_adjustment=False,
    clustering_adjustment=False
)
testing_results

Testing quasilier specification

In [ ]:
import pyblp
import numpy as np
import pandas as pd

pyblp.options.digits = 2
pyblp.options.verbose = False
pyblp.__version__

In [ ]:
product_data = pd.read_csv(pyblp.data.BLP_PRODUCTS_LOCATION)
product_data.head()
agent_data = pd.read_csv(pyblp.data.BLP_AGENTS_LOCATION)
agent_data.head()

In [ ]:
product_formulations = (
   pyblp.Formulation('1 + hpwt + air + mpd + space'),
   pyblp.Formulation('1 + prices + hpwt + air + mpd + space'),
   pyblp.Formulation('1 + log(hpwt) + air + log(mpg) + log(space) + trend')
)
product_formulations

In [ ]:
agent_formulation = pyblp.Formulation('0 + I(1 / income)')
agent_formulation

In [ ]:
problem = pyblp.Problem(product_formulations, product_data, agent_formulation, agent_data, costs_type='log')
problem

In [ ]:
initial_sigma = np.diag([3.612, 0, 4.628, 1.818, 1.050, 2.056])
initial_pi = np.c_[[0, -43.501, 0, 0, 0, 0]]

In [ ]:
results = problem.solve(
    initial_sigma,
    initial_pi,
    costs_bounds=(0.001, None),
    W_type='clustered',
    se_type='clustered',
    initial_update=True,
)
results

In [ ]:
ZD = results.problem.products.ZD
WD = results.updated_W
h = results.moments
h_i = ZD * results.xi
K2 = results.problem.K2
D = results.problem.D

XD = results.problem.products.X1  # NOTE: marco's code
XD_column_names = results.problem.products.dtype.fields['X1'][2]
price_in_linear_parameters = 'prices' in XD_column_names
if price_in_linear_parameters:
    XD = np.delete(XD, XD_column_names.index('prices'), 1)

if price_in_linear_parameters:    # NOTE: marco's code
    partial_y_theta = np.append(
        results.xi_by_theta_jacobian, -results.problem.products.prices, axis=1
    )
else:
    partial_y_theta = results.xi_by_theta_jacobian

# add price to the gradient
partial_y_theta = (np.append(
        results.xi_by_theta_jacobian, -results.problem.products.prices, axis=1
    ) if price_in_linear_parameters else results.xi_by_theta_jacobian)

# absorb fixed effects if they are specified
if results.problem.ED > 0:
    print("True")
    partial_y_theta = results.problem._absorb_demand_ids(partial_y_theta)
    partial_y_theta = np.reshape(
        partial_y_theta[0], [2256, len(results.theta) + int(price_in_linear_parameters)]
    )

# TODO: add note
from scipy.linalg import inv
if not XD.shape[1]:
    print("True")
    partial_xi_theta = partial_y_theta
else:
    print(XD.shape)
    print(WD.shape)
    print(ZD.shape)
    product = XD @ inv(XD.T @ ZD @ WD @ ZD.T @ XD) @ (XD.T @ ZD @ WD @ ZD.T @ partial_y_theta)
    partial_xi_theta = partial_y_theta - product

# partial_y_theta = np.reshape(partial_y_theta[0], [2256, len(results.theta) + int(price_in_linear_parameters)])

print(np.shape(XD)[1])
print(XD.shape[1] == 0)
# partial_y_theta = (
#     np.append(results.xi_by_theta_jacobian, -results.problem.products.prices, axis=1)
# )
# results.problem._absorb_demand_ids(partial_y_theta)

In [ ]:
import pyRVtest

In [ ]:
testing_problem = pyRVtest.Problem(
    cost_formulation = (
        pyRVtest.Formulation('1 + log(hpwt) + air + log(mpg) + log(space) + trend')
    ),
    instrument_formulation = (
        pyRVtest.Formulation('0 + demand_instruments0 + demand_instruments1')
    ),
    model_formulations = (
        pyRVtest.ModelFormulation(model_downstream='bertrand', ownership_downstream='firm_ids'),
        pyRVtest.ModelFormulation(model_downstream='monopoly', ownership_downstream='firm_ids')
    ),
    product_data = product_data,
    demand_results = results
)
testing_problem

In [ ]:
testing_results = testing_problem.solve(
    demand_adjustment=True,
    clustering_adjustment=True
)
testing_results

In [ ]:
results.problem.ED